# Structure Property visualizer

This is the second of two deliverables for the SiSc-Lab2020 project.

Authors = 

Supervisors: Dr. Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

In [1]:
# Instructions by supervisors
a = '''
## Jens
You have to implement this notebook

In the end only Markup text and output results should be seen if one hides the code cells (hide_code extension)

That is can easily exported into a nice pdf file.

Also the amount of python code in this notebook should be minimal.

Rather export the functions you use from a different file.

optional dump query results in a file, from which results will be reread for speed, i.e cache results

______________

An example of what we in the end aim for your find here.

https://www.materialscloud.org/discover/mofs#mcloudHeader

Clicking at one of the prerendered plots will open an interactive bokeh app.

Code for this you find here: https://github.com/materialscloud-org/structure-property-visualizer/blob/master/figure/main.py

For you sisc project this is to much, so do not view this as something you have to deliver but something that with the help of your work
in this project we will build out of it for our purposes.

So do not worry about an app.
_________________
**Your tasks are as follows:**

1. Implement a general interactive bokeh scatter plots with linked histograms (see static version examples/MP_convergence_scf_clean_150_240.png). 

   We gladly help you with it. (this goes into helpers.py and you import it here for usage). This function should not contain any aiida methods.
2. Extract float data you find in certain 'Dict' nodes into a/several pandas object, which will be then the datasource for this notebook. 

   So this notebook should not directly depend on any aiida methods (beyond load_profile)

## Johannes

Nothing much to add.

I'd just have bonus: if all this works, one could think about some actual data analysis, like clustering analysis or dimension reduction as we learned 
in Data Analysis & Visualization. For example, do [PCA](https://blog.exploratory.io/an-introduction-to-principal-component-analysis-pca-with-2018-world-soccer-players-data-810d84a14eab) 
with the quantities magnetic moment, energy, band gap, fermi energy, (structure, core levels). Ie, are some of these quantities (linearly) correlated in a 
data subset (e.g. all output nodes)? Since the data is already preprocessed mostly, this should be relatively easy, just throw the data into 
[scikit-learn](https://scikit-learn.org/stable/).
'''

In [2]:
# Imports
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook

In [3]:
# python imports:
from collections import Counter
import time
import numpy as np
import pandas as pd
#from pprint import pprint

# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add further imports
# project imports:
from aiida_jutools.sisc_lab import helpers
#import helpers
from bokeh.io import output_notebook

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

This notebook/dashboard will visualize the contents from the database of profile default


# Subastk D2.a: Data acquisition

Task: check which output dict nodes returned by workflows, which had `StructureData` nodes as inputs are there in the database.

For example, for a successful `FleurScfWorkChain`, there are two return `Dict` nodes, one is linked with
`last_fleur_calc_output` and one with `output_scf_wc_para`.

If a `StructureData` is an input of such a workflow you can extract the formula, `uuid` and other information you need from the 
`StructureData` which is always linked into workflows via the link name `structure`.

All the user should have to say:
```python
source = generate_structure_property_pandas_source('<workflow_name>')
```


In [6]:
#Find out valid plugin tools
# !verdi plugin list aiida.workflows

#Have a look at the attributes of dict nodes and sturcture nodes
# dict_project=['attributes','uuid']
# workflowdictlst = helpers.get_structure_workflow_dict(dict_project=dict_project)
#or
# structure_project=['uuid', 'extras','attributes']
# workflowdictlst = helpers.get_structure_workflow_dict(structure_project=structure_project)

In [7]:
# workflow_name = 'fleur_scf_wc' # Filter workflow
# workflowdictlst = helpers.get_structure_workflow_dict(workflow_filters={'attributes.process_label':workflow_name})
#or
workflow_name = None # No restriction. Querying by default
workflowdictlst = helpers.get_structure_workflow_dict()

print(len(workflowdictlst))
workflowdictlst[:2]

232


[{'structure': ['92ed2ebc-58db-46da-8d9d-6fbaff143e1e', 'Si2'],
  'workflow': ['a69a9c35-9cc3-4c78-9f4d-b1386638ec06', 'FleurScfWorkChain'],
  'dict': ['1bf803c6-342d-45d5-ba2d-707bea5aa909']},
 {'structure': ['92ed2ebc-58db-46da-8d9d-6fbaff143e1e', 'Si2'],
  'workflow': ['a69a9c35-9cc3-4c78-9f4d-b1386638ec06', 'FleurScfWorkChain'],
  'dict': ['9a18bbaf-a0e0-4a7c-83d5-0567f8520ef3']}]

In [8]:
# Some dict attributes: force, energy, total_energy, fermi_energy, bandgap, charge_density, distance_charge, 
#                       workflow_version, parser_info, material,
# Remarks:  workflow_version=0.2.2: energy, force
#           workflow_version=0.4.2: total_energy, material. 
#           parser: fermi_energy, bandgap.

dict_project=['uuid', 'attributes.force', 'attributes.energy', 'attributes.total_energy', 'attributes.distance_charge',             'attributes.bandgap', 'attributes.workflow_version', 'attributes.parser_info', 'attributes.material',
        'attributes.fermi_energy']
dictpd = helpers.generate_dict_property_pandas_source(
            workflow_name, 
            dict_project=dict_project, 
            filename='dict_property.json')
dictpd.head(5)

,dict_uuid,force,energy,total_energy,distance_charge,bandgap,workflow_version,parser_info,material,fermi_energy
0,1bf803c6,None,NaN,-580.066108,0.000206,NaN,0.4.2,None,Si2,NaN
1,9a18bbaf,None,-15784.402906,None,None,2.280895,None,AiiDA Fleur Parser v0.3.2,None,0.178967
2,89d7930c,None,NaN,None,10000,NaN,0.4.3,None,Si2,NaN
3,99bf83aa,None,-34634.006550,None,None,0.000000,None,AiiDA Fleur Parser v0.3.2,None,0.366738
4,1b70d4a0,None,NaN,-1272.776265,0.000056,NaN,0.4.2,None,Fe,NaN


In [9]:
structure_project=['uuid', 'extras.formula']
structurepd = helpers.generate_structure_property_pandas_source(
            workflow_name, 
            structure_project=structure_project,
            filename='structure_property.json')
structurepd.head(5)

,structure_uuid,formula
0,92ed2ebc,Si2
1,92ed2ebc,Si2
2,92ed2ebc,Si2
3,e02ebe7b,Fe
4,e02ebe7b,Fe


In [10]:
# Generate the combination of the above
combinepd = helpers.generate_combination_property_pandas_source(
            workflow_name, 
            dict_project=dict_project, 
            structure_project=structure_project,
            filename='combined_property.json')
combinepd.head(5)

,dict_uuid,force,energy,total_energy,distance_charge,bandgap,workflow_version,parser_info,material,fermi_energy,structure_uuid,formula
0,1bf803c6,None,NaN,-580.066108,0.000206,NaN,0.4.2,None,Si2,NaN,92ed2ebc,Si2
1,9a18bbaf,None,-15784.402906,None,None,2.280895,None,AiiDA Fleur Parser v0.3.2,None,0.178967,92ed2ebc,Si2
2,89d7930c,None,NaN,None,10000,NaN,0.4.3,None,Si2,NaN,92ed2ebc,Si2
3,99bf83aa,None,-34634.006550,None,None,0.000000,None,AiiDA Fleur Parser v0.3.2,None,0.366738,e02ebe7b,Fe
4,1b70d4a0,None,NaN,-1272.776265,0.000056,NaN,0.4.2,None,Fe,NaN,e02ebe7b,Fe


# Subtask D2.b: Interactive plot

Allow the user to choose, which properties to plot on what axis.

```python
xdata = source['distance']
ydata = source['energy']
```

Single bokeh scatter plot with histpgrams on both sides, hover tool should show 'input structure, formula, 
structure_uuid and dictnode uuid', as long as this information is available.

```python
bokeh_struc_prop_vis(xdata, ydata, src=source, **kwargs)
```

In [11]:
from helpers import read_json_file
df, xdata, ydata = read_json_file('combined_property.json','energy', 'bandgap')
df

,dict_uuid,force,energy,total_energy,distance_charge,bandgap,workflow_version,parser_info,material,fermi_energy,structure_uuid,formula
0,9a18bbaf,None,-15784.402906,None,None,2.280895,None,AiiDA Fleur Parser v0.3.2,None,0.178967,92ed2ebc,Si2
1,99bf83aa,None,-34634.006550,None,None,0.000000,None,AiiDA Fleur Parser v0.3.2,None,0.366738,e02ebe7b,Fe
2,09befb88,None,-34634.006550,None,None,0.000000,None,AiiDA Fleur Parser v0.3.2,None,0.366738,e02ebe7b,Fe
3,64b2c337,None,-34634.674870,None,None,0.000263,None,AiiDA Fleur Parser v0.3.2,None,0.363295,e02ebe7b,Fe
4,428b6d04,None,-28.136841,None,None,3.761806,None,AiiDA Fleur Parser v0.3.2,None,-0.286669,3c65ff7b,None
5,8fa0365c,None,-30.819362,None,None,10.694036,None,AiiDA Fleur Parser v0.3.2,None,-0.398062,56d0d2f5,None
6,9694613e,None,-1537.954893,None,None,5.609547,None,AiiDA Fleur Parser v0.3.2,None,-0.232118,b373b26b,None
7,3c4f5059,None,-34634.722937,None,None,0.009341,None,AiiDA Fleur Parser v0.3.2,None,0.361630,094d4847,None
8,8b15846f,None,-34634.716606,None,None,0.008352,None,AiiDA Fleur Parser v0.3.2,None,0.368299,50c6bf0c,None
9,98b319b6,None,-34634.710529,None,None,0.001951,None,AiiDA Fleur Parser v0.3.2,None,0.371747,3bb62161,None


In [12]:
from helpers import bokeh_struc_prop_vis
bokeh_struc_prop_vis('combined_property.json', 'energy', 'bandgap')